<a href="https://colab.research.google.com/github/tbrar/CPSC-501-A4-TensorFlow/blob/master/CHDModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [3]:
from google.colab import files
uploaded = files.upload()

Saving heart_train.csv to heart_train.csv


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [0]:
train_file_path = "heart_train.csv"
test_file_path = "heart_test.csv"

In [31]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

SELECT_COLUMNS = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'chd']
DEFAULTS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

example_batch, labels_batch = next(iter(temp_dataset)) 
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

example_batch, labels_batch = next(iter(temp_dataset)) 



[[126.     4.6    7.4   31.99  57.    28.67   0.37  60.  ]
 [112.     0.6    5.28  25.71  55.    27.02  27.77  38.  ]
 [138.     0.     3.96  24.7   53.    23.8    0.    45.  ]
 [128.     5.4    2.36  12.98  51.    18.36   6.69  61.  ]
 [164.     0.5    6.95  39.64  47.    41.76   3.81  46.  ]]

[1 1 0 0 1]


In [32]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

example_batch, labels_batch = next(iter(packed_train_data)) 

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,139.088384,3.534470,4.721641,25.417854,52.611111,25.980404,17.155328,42.727273
std,20.883153,4.622259,1.969109,7.862184,9.908200,4.185559,24.696052,14.758524
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000
25%,125.500000,0.050000,3.300000,19.825000,46.000000,22.757500,0.510000,31.000000
50%,134.000000,1.800000,4.325000,26.115000,53.000000,25.780000,7.780000,44.500000
75%,148.500000,5.152500,5.632500,31.102500,59.000000,28.402500,23.737500,55.250000
max,218.000000,31.200000,14.160000,42.490000,77.000000,45.720000,147.190000,64.000000


In [33]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()


array([[ 0.4267373 ,  1.8747394 , -0.4731286 ,  1.1106516 ,  0.44295526,
         0.09546999, -0.11156962,  0.9670837 ],
       [ 1.2886761 , -0.6348562 , -1.1688749 ,  1.0953883 ,  0.03924924,
         0.23404168,  1.4919256 ,  1.170356  ],
       [-0.33943048, -0.33197397, -1.0266786 , -0.48941293, -0.26353028,
         0.47056964, -0.31970003, -0.38806537],
       [-0.6267434 , -0.76466286, -0.08716703,  0.47851157, -0.46538326,
        -0.24856977,  0.7630641 , -0.11703555],
       [ 1.67176   , -0.00745731,  0.27340224, -0.43853655, -1.6765013 ,
        -0.9414282 , -0.35735786,  1.1025987 ]], dtype=float32)

In [34]:
CATEGORIES = {
    'famhist': ['Absent', 'Present'],
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

# See what you just created.
categorical_columns

categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])


[1. 0.]


In [35]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])


[ 1.          0.          0.4267373   1.8747394  -0.4731286   1.1106516
  0.44295526  0.09546999 -0.11156962  0.9670837 ]


In [37]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

model.fit(train_data, epochs=16)

test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

predictions = model.predict(test_data)

# Show some results
for prediction, chd in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted chd: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("Has CHD" if bool(chd) else "Doesn't have CHD"))


Epoch 1/16
80/80 [==============================] - 1s 16ms/step - loss: 1.5937 - accuracy: 0.7045
Epoch 2/16
80/80 [==============================] - 0s 3ms/step - loss: 1.0303 - accuracy: 0.7500
Epoch 3/16
80/80 [==============================] - 0s 3ms/step - loss: 0.8015 - accuracy: 0.7475
Epoch 4/16
80/80 [==============================] - 0s 3ms/step - loss: 0.7047 - accuracy: 0.7551
Epoch 5/16
80/80 [==============================] - 0s 3ms/step - loss: 0.6428 - accuracy: 0.7551
Epoch 6/16
80/80 [==============================] - 0s 3ms/step - loss: 0.6050 - accuracy: 0.7500
Epoch 7/16
80/80 [==============================] - 0s 3ms/step - loss: 0.5866 - accuracy: 0.7677
Epoch 8/16
80/80 [==============================] - 0s 3ms/step - loss: 0.5694 - accuracy: 0.7576
Epoch 9/16
80/80 [==============================] - 0s 3ms/step - loss: 0.5618 - accuracy: 0.7525
Epoch 10/16
80/80 [==============================] - 0s 3ms/step - loss: 0.5544 - accuracy: 0.7626
Epoch 11/16
80/80 